In [3]:
## In the second test I want to extract the symmetry matrices from the QE output or XML for the structures that the calculations run initially.
## Then, to compare them with the the symmetry matrices that are calculated with spglib for the structures before and after the relaxation.
## Lastly, I need to check again with the structures that are the calculations failed due to symmetry issues.

In [4]:
from aiida.orm import load_node, load_group, Group
import aiida 
import spglib
import numpy as np
from collections import defaultdict

aiida.load_profile()

Profile<uuid='960a408e945248238ac64a4982cfc6d7' name='dev'>

In [5]:
# Calculate the symmetry matrices with spglib
def get_symmetry_matrices_spglib(structure_node):
    
    structure_ase = structure_node.get_ase()
    
    lattice = structure_ase.cell.array
    positions = structure_ase.get_scaled_positions()
    numbers = structure_ase.get_atomic_numbers()
    
    structure_spglib = (lattice, positions, numbers)
    symmetry_dataset = spglib.get_symmetry(structure_spglib)
    rotations = symmetry_dataset['rotations']
    
    unique_rotations = {tuple(map(tuple, rotation)) for rotation in rotations}
    
    return unique_rotations, len(unique_rotations)

# extract the symmetry matrices from QE
def get_crystal_symmetries_qe(output_lines):
    import re

    symmetries = set()
    pattern = re.compile(r'\(\s*(-?\d+)\s*(-?\d+)\s*(-?\d+)\s*\)')

    collecting = False
    current_matrix = []

    for line in output_lines:
        
        if 'cryst.   s(' in line:
            #print(line)
            collecting = True
            current_matrix = []
        
        if collecting:
            match = pattern.search(line)
            if match:
                current_matrix.append(tuple(map(int, match.groups())))
                if len(current_matrix) == 3:
                    symmetries.add(tuple(current_matrix))
                    collecting = False

    return symmetries

# extract the WorkChainNodes from the descendants.
def get_workchain_node_ids(node_strings):

    workchain_ids = []
    
    for node_str in node_strings:
        if 'WorkChainNode' in node_str.node_type:
            # Extract the primary key (pk) from the string
            pk = node_str.pk
            workchain_ids.append(pk)
    
    return workchain_ids


def matrices_are_equal(set1, set2):
    identity_matrix = np.eye(3)
    
    # Create copies of the sets to work with
    set1_copy = set(set1)
    set2_copy = set(set2)
    
    # Collect matrices to remove after iteration
    to_remove_from_set1 = set()
    to_remove_from_set2 = set()
    
    for matrix1 in set1_copy: 
        matrix1_np = np.array(matrix1)  
        for matrix2 in set2_copy: 
            matrix2_np = np.array(matrix2)  
            
            matrix2_T = matrix2_np.T
            res = matrix1_np @ matrix2_T
            if np.array_equal(res, identity_matrix):
                to_remove_from_set1.add(matrix1)
                to_remove_from_set2.add(matrix2)
                break  
    
    # Remove matrices from the copies after the iteration
    set1_copy.difference_update(to_remove_from_set1)
    set2_copy.difference_update(to_remove_from_set2)
    
    # Check if both copies are empty
    return not set1_copy and not set2_copy


In [6]:
## First we can check the workchain nodes that the calculations were succesfull.

group_name = 'workchain/test/relax'
group = load_group(group_name)

successful_nodes = []

for node in group.nodes:
    workchain_node = load_node(node.pk)
    if node.is_finished:
        if node.is_finished_ok:
            successful_nodes.append(node.pk)

## Succesful Calculations

In [7]:
# Tests for the structures before the first relaxation. 

for node in successful_nodes:
    
    workflow_node = load_node(node)
    input_workchain = get_workchain_node_ids(workflow_node.called_descendants)

    workchain = load_node(min(input_workchain))
    retrieved = workchain.outputs.retrieved
    
    qe_aiida_out = retrieved.base.repository.get_object_content('aiida.out').split('\n')
    
    symmetries_qe = get_crystal_symmetries_qe(qe_aiida_out)
    
    structure = workchain.inputs.pw.structure
    
    symmetries_spglib, number_of_symmetries = get_symmetry_matrices_spglib(structure)
    
    if not matrices_are_equal(symmetries_qe, symmetries_spglib):
        print(node)
        print('The number of symmetries from QE : ', len(symmetries_qe))
        print('The number of symmetries from Spglib : ', len(symmetries_spglib))
        

In [8]:
# Tests for the structures before the second relaxation. 

for node in successful_nodes:
    
    workflow_node = load_node(node)
    input_workchain = get_workchain_node_ids(workflow_node.called_descendants)

    workchain = load_node(max(input_workchain))
    retrieved = workchain.outputs.retrieved
    
    qe_aiida_out = retrieved.base.repository.get_object_content('aiida.out').split('\n')
    
    symmetries_qe = get_crystal_symmetries_qe(qe_aiida_out)
    
    structure = workchain.inputs.pw.structure
    
    symmetries_spglib, number_of_symmetries = get_symmetry_matrices_spglib(structure)
    
    if not matrices_are_equal(symmetries_qe, symmetries_spglib):
        print(node)
        print('The number of symmetries from QE : ', len(symmetries_qe))
        print('The number of symmetries from Spglib : ', len(symmetries_spglib))

In [9]:
# Test the symmetry matrices for the structures that are produced from the relaxations. 


group_name = 'QE/test/symmetry'
group = load_group(group_name)

successful_nodes = [node.pk for node in group.nodes]


for node in successful_nodes:
    
    workflow_node = load_node(node)
    input_workchain = get_workchain_node_ids(workflow_node.called_descendants)[0]
    input_workchain_node = load_node(input_workchain)
    
    workchain = load_node(input_workchain_node.called_descendants[-1].pk)
    try:
        retrieved = workchain.outputs.retrieved
    except: 
        workchain = load_node(input_workchain_node.called_descendants[-2].pk)
        retrieved = workchain.outputs.retrieved
        
        
    qe_aiida_out = retrieved.base.repository.get_object_content('aiida.out').split('\n')
    
    symmetries_qe = get_crystal_symmetries_qe(qe_aiida_out)
    
    structure = workchain.inputs.structure
    
    symmetries_spglib, number_of_symmetries = get_symmetry_matrices_spglib(structure)
    
    if not matrices_are_equal(symmetries_qe, symmetries_spglib):
        print(node)
        print('The number of symmetries from QE : ', len(symmetries_qe))
        print('The number of symmetries from Spglib : ', len(symmetries_spglib))

## Failed Calculations

In [10]:
group_name = 'workchain/test/relax'
group = load_group(group_name)

# Initialize counters and structures
total_finished = 0
successful_finished = 0
non_successful_finished = 0
non_successful_nodes = []
nodes_errors = defaultdict(list)
status_messages = {}
kpoint_failed = set()

# Iterate over nodes in the group
for node in group.nodes:
    workchain_node = load_node(node.pk)
    if workchain_node.is_finished:
        total_finished += 1
        if workchain_node.is_finished_ok:
            successful_finished += 1
        else:
            non_successful_finished += 1
            print(f"Calculation {node.pk} was NOT successful", end='\r')
            non_successful_nodes.append(workchain_node)
            nodes_errors[workchain_node.exit_status].append(workchain_node)



for status, workchains in nodes_errors.items():
    for workchain in workchains:
        for calculation in workchain.called_descendants:
            calc = load_node(calculation.pk)
            if calc.exit_status is not None:
                message = calc.exit_message
                if calc.exit_status == 541:
                    kpoint_failed.add(workchain)
                if calc.exit_status not in status_messages:
                    status_messages[calc.exit_status] = message
                elif message != status_messages[calc.exit_status]:
                    # Handle different messages for the same status if needed
                    pass

In [11]:
nodes_errors

defaultdict(list,
            {403: [<WorkChainNode: uuid: c3babc18-16a6-4065-a797-6140f4c86dac (pk: 74663) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: 7cf3c6f2-09aa-4be4-97b4-9c6759f2b9c7 (pk: 75892) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: ca5d4c89-27f0-4eb2-85fe-6a19ec4a7398 (pk: 75961) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: cb12085d-30b3-4fe0-8fc4-420674b1eacd (pk: 76368) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: 6cea05aa-fe68-4a3a-9506-8e9e93405fc9 (pk: 79909) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: 3f9daadd-f0e2-42d0-b2de-fe294816fa0a (pk: 80094) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: 9277edc9-c8df-4aa4-be21-333c27a211d0 (pk: 80826) (aiida.workflows:quantumespresso.pw.relax)>,
              <WorkChainNode: uuid: 845df396-0002-45e3-9cbe-2e838d0

In [12]:
for node in list(kpoint_failed):
    print(node)
    workflow_node = load_node(node.pk)
    input_workchain = get_workchain_node_ids(workflow_node.called_descendants)[0]
    
    workchain = load_node(input_workchain)
    pw_node = load_node(workchain.called_descendants[-1].pk)
    print(pw_node)
    retrieved = pw_node.outputs.retrieved
    qe_aiida_out = retrieved.base.repository.get_object_content('aiida.out').split('\n')
    
    symmetries_qe = get_crystal_symmetries_qe(qe_aiida_out)
    
    structure = workchain.inputs.pw.structure
    
    symmetries_spglib, number_of_symmetries = get_symmetry_matrices_spglib(structure)
    print(len(symmetries_qe),len(symmetries_spglib))
    if not matrices_are_equal(symmetries_qe, symmetries_spglib):
        print(node)
        print('The number of symmetries from QE : ', len(symmetries_qe))
        print('The number of symmetries from Spglib : ', len(symmetries_spglib))

uuid: 0647d276-c050-4acd-9993-6f0e9c8e6e9a (pk: 91425) (aiida.workflows:quantumespresso.pw.relax)
uuid: d47560b8-004c-4c1c-b430-bb863019504d (pk: 91718) (aiida.calculations:quantumespresso.pw)
12 12
uuid: 9a7cb21e-9d41-4345-bb6b-c831ba56145e (pk: 85103) (aiida.workflows:quantumespresso.pw.relax)
uuid: 97590d06-6df7-4065-84d0-f99a941bce48 (pk: 85511) (aiida.calculations:quantumespresso.pw)
12 12
uuid: cb12085d-30b3-4fe0-8fc4-420674b1eacd (pk: 76368) (aiida.workflows:quantumespresso.pw.relax)
uuid: a6e3e30f-5092-4649-8c50-2332c1560900 (pk: 76461) (aiida.calculations:quantumespresso.pw)
12 12


In [13]:
# Save the structures that failed to run in the kpoints_problem group to test them again. 

pk_to_add = [91425,85103,76368]

group_name = 'kpoints_problem'


group, created = Group.objects.get_or_create(label=group_name)



for node in pk_to_add: 
    workchain_node = load_node(node)
    
    structure = workchain_node.inputs.structure

    group.add_nodes(structure)

/tmp/ipykernel_3434044/1417270559.py:8: AiidaDeprecationWarning: `objects` property is deprecated, use `collection` instead. (this will be removed in v3)
  group, created = Group.objects.get_or_create(label=group_name)
